In [1]:
import glob
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler
import implicit

from implicit import approximate_als
import nmslib
global path
path='/home/user/AI_Works/coin/Recommendation/'
random.seed(100)
np.random.seed(100)
OPENBLAS_NUM_THREADS = 1
RESULTS=pd.DataFrame()
global sparse_user_item
global sparse_item_user
global item_lookup
global user_lookup
global model

In [2]:
def consumed(user_id):
    global item_lookup
    global user_lookup
    global sparse_user_item
    global sparse_item_user
    consumed_idx = sparse_user_item[user_id,:].nonzero()[1].astype(str)

    consumed_items = item_lookup.itemcode.loc[item_lookup.item_id.isin(consumed_idx)]
    return (consumed_items.tolist())

In [3]:
def recommended(user_id):
    global item_lookup
    global user_lookup
    global sparse_user_item
    global sparse_item_user
    global model
    global data
    recommended = model.recommend(user_id, sparse_user_item)

    items = []
    scores = []


    for item in recommended:
        idx, score = item
        items.append(data.itemcode.loc[data.item_id == idx].iloc[0])
        scores.append(score)
        recommendations = pd.DataFrame({'item': items, 'score': scores})
    return items
    #print "recommended items: \n",items

In [4]:
def all_clients_recommendation(filename,client_id):
    global sparse_user_item
    global sparse_item_user
    global item_lookup
    global user_lookup
    global model
    global data
    global path
    df = pd.read_csv(filename,usecols=['CardNo','productname','itemcode','quantity'])
    data = df.dropna()
    data.columns=['user','item','itemcode','quantity']
    data = data.copy()
    data['user'] = data['user'].astype("category")
    data['item'] = data['item'].astype("category")
    data['user_id'] = data['user'].cat.codes
    data['item_id'] = data['item'].cat.codes
    item_lookup = data[['item_id', 'item','itemcode']].drop_duplicates()
    item_lookup['item_id'] = item_lookup.item_id.astype(str)
    item_lookup['itemcode'] = item_lookup.itemcode.astype(str)

    user_lookup = data[['user_id', 'user']].drop_duplicates()
    user_lookup['user_id'] = user_lookup.user_id.astype(str)

    sparse_item_user = sparse.csr_matrix((data['quantity'].astype(float), (data['item_id'], data['user_id'])))
    sparse_user_item = sparse.csr_matrix((data['quantity'].astype(float), (data['user_id'], data['item_id'])))

    model = implicit.approximate_als.NMSLibAlternatingLeastSquares()

    alpha_val = 15
    data_conf = (sparse_item_user * alpha_val).astype('double')
    # Fit the model
    model.fit(data_conf)
    user_lookup['user_id'] = user_lookup['user_id'].apply(np.int64)
    user_lookup['consumed'] = user_lookup.apply(lambda row: consumed(row['user_id']), axis=1)
    user_lookup['recommended'] = user_lookup.apply(lambda row: recommended(row['user_id']), axis=1)
    user_lookup.columns=['User_id','CardNo','Consumed items','Recommended items']
    user_lookup=user_lookup.reset_index(drop=True)
    user_lookup=user_lookup.drop(['User_id'], axis=1)
    user_lookup['Client_ID']=client_id
    user_lookup.to_csv(path+'recommendations_'+client_id+'.csv')
    return user_lookup

    
    

In [5]:
import re
for name in glob.glob('/home/user/AI_Works/coin/data/received/*_received.csv'):
    client_id=re.findall("\d{4}",name)[0]
    RESULTS=RESULTS.append(all_clients_recommendation(name,client_id),ignore_index=True)
    
print RESULTS
   
    

100%|██████████| 15.0/15 [00:03<00:00,  4.91it/s]
100%|██████████| 15.0/15 [00:18<00:00,  1.25s/it]


             CardNo                                     Consumed items  \
0      7.000000e+14  [TAC2, TAC5, TAC10, TAC17, TAC33, TAC23, TAC28...   
1      7.000000e+14  [TAC2, TAC5, TAC9, TAC10, TAC16, TAC22, TAC38,...   
2      7.000000e+14  [TAC2, TAC6, TAC15, TAC17, TAC19, TAC42, TAC23...   
3      7.000000e+14  [TAC2, TAC5, TAC9, TAC10, TAC15, TAC16, TAC17,...   
4      7.000000e+14  [TAC2, TAC6, TAC9, TAC17, TAC31, TAC38, TAC44,...   
5      7.000000e+14                               [TAC2, TAC31, TAC12]   
6      7.000000e+14                                [TAC2, TAC9, TAC38]   
7      7.000000e+14                        [TAC2, TAC17, TAC28, TAC40]   
8      7.000000e+14                 [TAC2, TAC16, TAC33, TAC38, TAC18]   
9      7.000000e+14                         [TAC2, TAC4, TAC13, TAC34]   
10     7.000000e+14     [TAC2, TAC16, TAC31, TAC21, TAC4, TAC36, TAC3]   
11     7.000000e+14   [TAC2, TAC31, TAC42, TAC45, TAC12, TAC26, TAC50]   
12     7.000000e+14                   

In [12]:
from time import gmtime, strftime
date=strftime("%Y-%m-%d", gmtime())
RESULTS['date']=date
RESULTS=RESULTS[['date','Client_ID','CardNo','Consumed items','Recommended items']]
RESULTS.to_csv(path+'All_clients_recommendations.csv')

In [11]:
RESULTS

,Client_ID,CardNo,Consumed items,Recommended items,date
0,2211,7.000000e+14,"[TAC2, TAC5, TAC10, TAC17, TAC33, TAC23, TAC28...","[TAC16, TAC13, TAC22, TAC44, TAC43, TAC40, TAC...",2018-08-28
1,2211,7.000000e+14,"[TAC2, TAC5, TAC9, TAC10, TAC16, TAC22, TAC38,...","[TAC44, TAC50, TAC40, TAC19, TAC49, TAC29, TAC...",2018-08-28
2,2211,7.000000e+14,"[TAC2, TAC6, TAC15, TAC17, TAC19, TAC42, TAC23...","[TAC24, TAC16, TAC46, TAC38, TAC22, TAC20, TAC...",2018-08-28
3,2211,7.000000e+14,"[TAC2, TAC5, TAC9, TAC10, TAC15, TAC16, TAC17,...","[TAC21, TAC35, TAC18, TAC6, TAC47, TAC12, TAC2...",2018-08-28
4,2211,7.000000e+14,"[TAC2, TAC6, TAC9, TAC17, TAC31, TAC38, TAC44,...","[TAC10, TAC21, TAC8, TAC25, TAC24, TAC26, TAC4...",2018-08-28
5,2211,7.000000e+14,"[TAC2, TAC31, TAC12]","[TAC21, TAC25, TAC47, TAC26, TAC17, TAC5, TAC2...",2018-08-28
6,2211,7.000000e+14,"[TAC2, TAC9, TAC38]","[TAC14, TAC44, TAC5, TAC35, TAC36, TAC3, TAC26...",2018-08-28
7,2211,7.000000e+14,"[TAC2, TAC17, TAC28, TAC40]","[TAC36, TAC20, TAC39, TAC26, TAC44, TAC10, TAC...",2018-08-28
8,2211,7.000000e+14,"[TAC2, TAC16, TAC33, TAC38, TAC18]","[TAC44, TAC14, TAC49, TAC5, TAC20, TAC3, TAC46...",2018-08-28
9,2211,7.000000e+14,"[TAC2, TAC4, TAC13, TAC34]","[TAC36, TAC49, TAC5, TAC39, TAC37, TAC50, TAC4...",2018-08-28
